In [1]:
# load libraries
import pandas as pd
from pandas import DataFrame as df
import pickle
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import xgboost as xgb
import matplotlib.pyplot as plt
import numpy as np
from imblearn.over_sampling import RandomOverSampler

# local module with helper utils
import model_utils as mutils
from model_utils.evaluation import evaluate_model, get_metrics

current_k_fold=9
# set seed
SEED=current_k_fold**3
np.random.seed(SEED)

# pandas options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# deserialize pre-processed data
path_to_pickle = f'../data/kddcup/kdd_preprocessed_k{current_k_fold}.pkl'

with open(path_to_pickle, 'rb') as f:
    data = pickle.load(f)
    X_train = data['X_train']
    y_train = data['y_train']

    X_val = data['X_val']
    y_val = data['y_val']

    X_test = data['X_test']
    y_test = data['y_test']

    le = data['label_enc']
    oh_enc = data['oh_enc']


print('Data loaded successfully')

# get imbalance ratio for each data set
IR_train = mutils.imb_ratio(y_train.value_counts())
IR_val = mutils.imb_ratio(y_val.value_counts())
IR_test = mutils.imb_ratio(y_test.value_counts())

# print imbalance ratios. They should be (nearly) the same. 
print(f"Imbalance ratio in training data: {IR_train}")
print(f"Imbalance ratio in validation data: {IR_val}")
print(f"Imbalance ratio in test data: {IR_test}")

"""
## Oversample minority class for training only
"""

ros = RandomOverSampler(random_state=SEED, sampling_strategy=1)
X_train, y_train = ros.fit_resample(X_train.to_numpy(), y_train.to_numpy())

res_value_counts = df(y_train).value_counts()

print("New Imbalance ratio:", mutils.imb_ratio(res_value_counts))

Data loaded successfully
Imbalance ratio in training data: 4.13
Imbalance ratio in validation data: 4.13
Imbalance ratio in test data: 4.13
New Imbalance ratio: 1.0


# Hyperparameter finding and tuning

## Grid Tuning

In [2]:
# xbg hyperparams grid search

# hyperparams grid; params from https://xgboost.readthedocs.io/en/latest/parameter.html & https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/#:~:text=to%20Booster%20parameters.-,Booster%20Parameters,-Though%C2%A0there%20are
param_grid = {
       'device': [mutils.get_device()], # specify the device to be used for XGBoost
    'objective': ['binary:logistic'], # logistic regression for binary classification, output probability
    'verbose': [3], # 0 (silent), 1 (warning), 2 (info), 3 (debug)

    'n_estimators': [150, 170, 200, 250],
    'booster': ['gbtree'],
    'tree_method': ['hist'],

    # tree booster params
    'max_leaves': [2**6, 2**8],
    'learning_rate': [0.3], # alias: eta, step size
    'gamma': [0.3], # min loss reduction to create new tree split
    'max_depth': [5,6, 7], # max depth of tree
    'subsample': [0.7],
   # 'colsample_bytree': [0.5, 0.7, 1], # subsample ratio of columns when constructing each tree
    'reg_alpha': [1], # l1 regularization term on weights
    'reg_lambda': [0.3], # l2 regularization term on weights
    'scale_pos_weight': [2,4], # control balance of positive and negative weights, useful for unbalanced classes. recommended to start with sum(negative instances) / sum(positive instances)
   
   }

clf = XGBClassifier(random_state=SEED, verbose=1, device=mutils.get_device())

# search  = GridSearchCV(clf, param_grid, scoring='roc_auc', verbose=1)
# search.fit(X_train, y_train)

# -> executed on Google Colab on A100 GPU, took ~ 37 minutes
# load clf from pickle
# clf_pkl = 'clf_xgb_cc.pkl'

# with open(clf_pkl, 'rb') as f:
#     data = pickle.load(f)
#     search = data['search']
#     results_df = data['results']

# results_df.head()

## RandomSearch Tuning

In [11]:
# search space

# hyperparams grid; params from https://xgboost.readthedocs.io/en/latest/parameter.html & https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/#:~:text=to%20Booster%20parameters.-,Booster%20Parameters,-Though%C2%A0there%20are
search_space_random_tuned_1= {
     # general params
    'device': [mutils.get_device()], # specify the device to be used for XGBoost
    'objective': ['binary:logistic'], # logistic regression for binary classification, output probability

    # tree booster params
    'max_leaves': [0, 2**6, 2**8], # default 0
    'learning_rate': [0.002, 0.1, 0.3], # alias: eta, step size
    'gamma': [0.1, 0.3], # min loss reduction to create new tree split
    'max_depth': [6, 12, 20, 25], # max depth of tree
    'subsample': [0.4, 0.7, 1],
    'colsample_bytree': [0.4, 0.5, 0.7, 1], # subsample ratio of columns when constructing each tree
    'reg_alpha': [0, 0.9, 1], # l1 regularization term on weights, default = 0
    'reg_lambda': [0.3, 1], # l2 regularization term on weights, default = 1
    'scale_pos_weight': [1, 2, 4, 580], # control balance of positive and negative weights, useful for unbalanced classes. recommended to start with sum(negative instances) / sum(positive instances)
    'eval_metric': ['auc'],
    'n_estimators': [100, 170, 500, 700, 1000, 2000, 5000],
    'booster': ['gbtree'],
    'tree_method': ['hist'],
}

clf = XGBClassifier(random_state=SEED, device=mutils.get_device())

random_search  = RandomizedSearchCV(clf, search_space_random_tuned_1, scoring='roc_auc', verbose=3, cv=5)
random_search.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END booster=gbtree, colsample_bytree=1, device=cpu, eval_metric=auc, gamma=0.3, learning_rate=0.002, max_depth=20, max_leaves=0, n_estimators=700, objective=binary:logistic, reg_alpha=0, reg_lambda=0.3, scale_pos_weight=4, subsample=1, tree_method=hist;, score=0.999 total time=  16.2s
[CV 2/5] END booster=gbtree, colsample_bytree=1, device=cpu, eval_metric=auc, gamma=0.3, learning_rate=0.002, max_depth=20, max_leaves=0, n_estimators=700, objective=binary:logistic, reg_alpha=0, reg_lambda=0.3, scale_pos_weight=4, subsample=1, tree_method=hist;, score=0.999 total time=  15.9s
[CV 3/5] END booster=gbtree, colsample_bytree=1, device=cpu, eval_metric=auc, gamma=0.3, learning_rate=0.002, max_depth=20, max_leaves=0, n_estimators=700, objective=binary:logistic, reg_alpha=0, reg_lambda=0.3, scale_pos_weight=4, subsample=1, tree_method=hist;, score=0.999 total time=  16.6s
[CV 4/5] END booster=gbtree, colsample_bytree=1, devic

RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           device=device(type='cpu'),
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None...
                                        'eval_metric': ['auc'],
                                        'gamma': [0.1, 0.3],
                                        'learning_rate': [0.002, 0.1, 0.3],
                                        'max_depth': [6, 12, 20, 25],
                                        'max_leaves': [0, 64, 256],
                                        'n_estimators': [100, 170, 500, 700,
                                                         1000, 2000, 5000],
                                        'objective': ['binary:logistic'],
                                        'reg_alpha': [0, 0.9, 1],
                                        'reg_lambda': [0.3, 1],
                                        'scale_pos_weight': [1, 2, 4, 580],
                                        'subsample': [0.4, 0.7, 1],
                                        'tree_method': ['hist']},
                   scoring='roc_auc', verbose=3)

In [ ]:
xgb_best = random_search.best_estimator_
xgb_best.get_params()

# Train the model

In [8]:
param_grid_search_tuned_1 = {
    # general params
    "device": mutils.get_device(),  # specify the device to be used for XGBoost
    "objective": "binary:logistic",  # logistic regression for binary classification, output probability,
    # tree booster params
    "max_leaves": 2**6,
    "eta": 0.1,  # alias: learning_rate, step size
    "gamma": 0.3,  # min loss reduction to create new tree split
    "max_depth": 5,  # max depth of tree
    "subsample": 0.7,
    # 'colsample_bytree': [0.5, 0.7, 1], # subsample ratio of columns when constructing each tree
    "reg_alpha": 1,  # l1 regularization term on weights
    "reg_lambda": 0.3,  # l2 regularization term on weights
    "scale_pos_weight": 2,  # control balance of positive and negative weights, useful for unbalanced classes. recommended to start with sum(negative instances) / sum(positive instances),
    "eval_metric": "auc",
    "n_estimators": 350,
    "booster": "gbtree",
    "tree_method": "hist",
}

param_grid_search_tuned_2 = {
    # general params
    "device": mutils.get_device(),  # specify the device to be used for XGBoost
    "objective": "binary:logistic",  # logistic regression for binary classification, output probability
    # tree booster params
    "max_leaves": 2**6,
    "learning_rate": 0.3,  # alias: eta, step size
    "gamma": 0.3,  # min loss reduction to create new tree split
    "max_depth": 5,  # max depth of tree
    "subsample": 0.7,
    # 'colsample_bytree': [0.5, 0.7, 1], # subsample ratio of columns when constructing each tree
    "reg_alpha": 1,  # l1 regularization term on weights
    "reg_lambda": 0.3,  # l2 regularization term on weights
    "scale_pos_weight": 2,  # control balance of positive and negative weights, useful for unbalanced classes. recommended to start with sum(negative instances) / sum(positive instances)
    "eval_metric": "auc",
    "n_estimators": 250,
    "booster": "gbtree",
    "tree_method": "hist",
}

param_random_search_tuned_1 = {
    "objective": "binary:logistic",
    "booster": "gbtree",
    "colsample_bytree": 0.4,
    "device": mutils.get_device(),
    "eval_metric": "auc",
    "gamma": 0.1,
    "learning_rate": 0.3,
    "max_depth": 20,
    "max_leaves": 64,
    "n_estimators": 170,
    "reg_alpha": 0.9,
    "reg_lambda": 0.3,
    "scale_pos_weight": 2,
    "subsample": 1,
    "tree_method": "hist",
}

param_random_search_tuned_2 = {
    "objective": "binary:logistic",
    "booster": "gbtree",
    "colsample_bytree": 0.7,
    "device": mutils.get_device(),
    "eval_metric": "auc",
    "gamma": 0.1,
    "learning_rate": 0.1,
    "max_depth": 20,
    "max_leaves": 256,
    "n_estimators": 1000,
    "reg_alpha": 1,
    "reg_lambda": 1,
    "scale_pos_weight": 4,
    "subsample": 1,
    "tree_method": "hist",
}


param_adjusted_vanilla_xgb = {
    # general params
    "device": mutils.get_device(),  # specify the device to be used for XGBoost
    "objective": "binary:logistic",  # logistic regression for binary classification, output probability
    "scale_pos_weight": 580,  # control balance of positive and negative weights, useful for unbalanced classes. recommended to start with sum(negative instances) / sum(positive instances)
    "eval_metric": "auc",
    "booster": "gbtree",
    "tree_method": "hist",
}

In [ ]:
# pandas.core.series.Series/Numpy to DataFrame
y_train = df(y_train)
y_val = df(y_val)
y_test = df(y_test)

# RandomOversampling removes column and returns numpy, reverse it here:
X_train = df(X_train)
X_train.columns = X_val.columns
eval_set = [(X_val, y_val)]

"""
Grid Tuned 1
"""
grid_tuned_1 = XGBClassifier(random_state=SEED, **param_grid_search_tuned_1)

grid_tuned_1.fit(X_train, y_train, eval_set=eval_set)

"""
Grid Tuned 2
"""
grid_tuned_2 = XGBClassifier(random_state=SEED, **param_grid_search_tuned_2)

grid_tuned_2.fit(X_train, y_train, eval_set=eval_set)


"""
Random Searched 1
"""
random_search_tuned_1 = XGBClassifier(random_state=SEED, **param_random_search_tuned_1)

random_search_tuned_1.fit(X_train, y_train, eval_set=eval_set)


"""
Random Searched 2
"""
random_search_tuned_2 = XGBClassifier(random_state=SEED, **param_random_search_tuned_2)

random_search_tuned_2.fit(X_train, y_train, eval_set=eval_set)
# 
"""
Vanilla XGB

- using default params: https://xgboost.readthedocs.io/en/stable/parameter.html#
"""
vanilla_xgb = XGBClassifier(random_state=SEED)

vanilla_xgb.fit(X_train, y_train, eval_set=eval_set)

"""
Slightly Adjusted Vanilla XGB
"""
adjusted_vanilla_xgb = XGBClassifier(random_state=SEED, **param_adjusted_vanilla_xgb)

adjusted_vanilla_xgb.fit(X_train, y_train, eval_set=eval_set)

In [21]:
print(f"current fold: {current_k_fold}")

current fold: 3


# fold 1

In [11]:
evaluate_model(
    [
        vanilla_xgb,
		
        adjusted_vanilla_xgb,
		
        random_search_tuned_1,
		
		random_search_tuned_2,
		
        grid_tuned_1,
		
        grid_tuned_2,
    ],
    X_test,
    y_test,
    names=[
        "Vanilla XGB",
		
        "Adjusted Vanilla XGB",
		
        "Random Searched Tuned 1",
		
		"Random Searched Tuned 2",
		
        "Grid Tuned Model 1",
		
        "Grid Tuned Model 2",
    ],
	as_table=True
)

,Model Name,AUCPRC,F1,G-Mean,MCC,Precision,Recall,ROCAUC,ACCURACY,TP,FP,TN,FN
0,Vanilla XGB,0.9942,0.9419,0.9843,0.9292,0.8918,0.9980,0.9985,0.9760,6047.0,734.0,24310.0,12.0
2,Random Searched Tuned 1,0.9942,0.9412,0.9845,0.9284,0.8894,0.9993,0.9985,0.9757,6055.0,753.0,24291.0,4.0
3,Random Searched Tuned 2,0.9942,0.9402,0.9844,0.9272,0.8873,0.9997,0.9985,0.9752,6057.0,769.0,24275.0,2.0
4,Grid Tuned Model 1,0.9942,0.9410,0.9845,0.9282,0.8891,0.9993,0.9986,0.9756,6055.0,755.0,24289.0,4.0
5,Grid Tuned Model 2,0.9941,0.9413,0.9844,0.9285,0.8899,0.9990,0.9985,0.9757,6053.0,749.0,24295.0,6.0
1,Adjusted Vanilla XGB,0.6846,0.6495,0.8594,0.5958,0.4813,0.9987,0.8612,0.7901,6051.0,6522.0,18522.0,8.0


# fold 2

In [17]:
evaluate_model(
    [
        vanilla_xgb,
		
        adjusted_vanilla_xgb,
		
        random_search_tuned_1,
		
		random_search_tuned_2,
		
        grid_tuned_1,
		
        grid_tuned_2,
    ],
    X_test,
    y_test,
    names=[
        "Vanilla XGB",
		
        "Adjusted Vanilla XGB",
		
        "Random Searched Tuned 1",
		
		"Random Searched Tuned 2",
		
        "Grid Tuned Model 1",
		
        "Grid Tuned Model 2",
    ],
	as_table=True
)

,Model Name,AUCPRC,F1,G-Mean,MCC,Precision,Recall,ROCAUC,ACCURACY,TP,FP,TN,FN
0,Vanilla XGB,0.9946,0.9415,0.9842,0.9288,0.8910,0.9982,0.9986,0.9759,6048.0,740.0,24304.0,11.0
2,Random Searched Tuned 1,0.9946,0.9398,0.9840,0.9268,0.8874,0.9988,0.9986,0.9751,6052.0,768.0,24276.0,7.0
3,Random Searched Tuned 2,0.9946,0.9388,0.9838,0.9255,0.8852,0.9992,0.9987,0.9746,6054.0,785.0,24259.0,5.0
4,Grid Tuned Model 1,0.9946,0.9398,0.9840,0.9267,0.8871,0.9990,0.9986,0.9751,6053.0,770.0,24274.0,6.0
5,Grid Tuned Model 2,0.9945,0.9397,0.9839,0.9266,0.8874,0.9987,0.9986,0.9751,6051.0,768.0,24276.0,8.0
1,Adjusted Vanilla XGB,0.9944,0.9373,0.9834,0.9238,0.8826,0.9992,0.9986,0.9740,6054.0,805.0,24239.0,5.0


# fold 3

In [22]:
evaluate_model(
    [
        vanilla_xgb,
		
        adjusted_vanilla_xgb,
		
        random_search_tuned_1,
		
		random_search_tuned_2,
		
        grid_tuned_1,
		
        grid_tuned_2,
    ],
    X_test,
    y_test,
    names=[
        "Vanilla XGB",
		
        "Adjusted Vanilla XGB",
		
        "Random Searched Tuned 1",
		
		"Random Searched Tuned 2",
		
        "Grid Tuned Model 1",
		
        "Grid Tuned Model 2",
    ],
	as_table=True
)

,Model Name,AUCPRC,F1,G-Mean,MCC,Precision,Recall,ROCAUC,ACCURACY,TP,FP,TN,FN
2,Random Searched Tuned 1,0.9948,0.9415,0.9847,0.9288,0.8899,0.9995,0.9987,0.9758,6056.0,749.0,24295.0,3.0
3,Random Searched Tuned 2,0.9948,0.9402,0.9843,0.9272,0.8875,0.9995,0.9987,0.9752,6056.0,768.0,24276.0,3.0
4,Grid Tuned Model 1,0.9948,0.9413,0.9846,0.9286,0.8895,0.9995,0.9987,0.9757,6056.0,752.0,24292.0,3.0
5,Grid Tuned Model 2,0.9948,0.9418,0.9848,0.9292,0.8905,0.9995,0.9987,0.9760,6056.0,745.0,24299.0,3.0
0,Vanilla XGB,0.9947,0.9426,0.9847,0.9301,0.8925,0.9987,0.9987,0.9763,6051.0,729.0,24315.0,8.0
1,Adjusted Vanilla XGB,0.9946,0.9391,0.9842,0.9260,0.8852,1.0000,0.9987,0.9747,6059.0,786.0,24258.0,0.0


# fold 4

In [25]:
evaluate_model(
    [
        vanilla_xgb,
		
        adjusted_vanilla_xgb,
		
        random_search_tuned_1,
		
		random_search_tuned_2,
		
        grid_tuned_1,
		
        grid_tuned_2,
    ],
    X_test,
    y_test,
    names=[
        "Vanilla XGB",
		
        "Adjusted Vanilla XGB",
		
        "Random Searched Tuned 1",
		
		"Random Searched Tuned 2",
		
        "Grid Tuned Model 1",
		
        "Grid Tuned Model 2",
    ],
	as_table=True
)

,Model Name,AUCPRC,F1,G-Mean,MCC,Precision,Recall,ROCAUC,ACCURACY,TP,FP,TN,FN
2,Random Searched Tuned 1,0.9950,0.9421,0.9845,0.9294,0.8916,0.9987,0.9987,0.9761,6052.0,736.0,24307.0,8.0
3,Random Searched Tuned 2,0.9950,0.9412,0.9845,0.9284,0.8894,0.9993,0.9987,0.9757,6056.0,753.0,24290.0,4.0
4,Grid Tuned Model 1,0.9950,0.9422,0.9847,0.9296,0.8915,0.9990,0.9987,0.9761,6054.0,737.0,24306.0,6.0
0,Vanilla XGB,0.9949,0.9431,0.9846,0.9306,0.8939,0.9980,0.9987,0.9765,6048.0,718.0,24325.0,12.0
5,Grid Tuned Model 2,0.9949,0.9424,0.9846,0.9298,0.8921,0.9987,0.9987,0.9762,6052.0,732.0,24311.0,8.0
1,Adjusted Vanilla XGB,0.9945,0.9394,0.9840,0.9263,0.8864,0.9992,0.9985,0.9749,6055.0,776.0,24267.0,5.0


# fold 5

In [27]:
evaluate_model(
    [
        vanilla_xgb,
		
        adjusted_vanilla_xgb,
		
        random_search_tuned_1,
		
		random_search_tuned_2,
		
        grid_tuned_1,
		
        grid_tuned_2,
    ],
    X_test,
    y_test,
    names=[
        "Vanilla XGB",
		
        "Adjusted Vanilla XGB",
		
        "Random Searched Tuned 1",
		
		"Random Searched Tuned 2",
		
        "Grid Tuned Model 1",
		
        "Grid Tuned Model 2",
    ],
	as_table=True
)

,Model Name,AUCPRC,F1,G-Mean,MCC,Precision,Recall,ROCAUC,ACCURACY,TP,FP,TN,FN
2,Random Searched Tuned 1,0.9950,0.9421,0.9845,0.9294,0.8916,0.9987,0.9987,0.9761,6052.0,736.0,24307.0,8.0
3,Random Searched Tuned 2,0.9950,0.9412,0.9845,0.9284,0.8894,0.9993,0.9987,0.9757,6056.0,753.0,24290.0,4.0
4,Grid Tuned Model 1,0.9950,0.9422,0.9847,0.9296,0.8915,0.9990,0.9987,0.9761,6054.0,737.0,24306.0,6.0
0,Vanilla XGB,0.9949,0.9431,0.9846,0.9306,0.8939,0.9980,0.9987,0.9765,6048.0,718.0,24325.0,12.0
5,Grid Tuned Model 2,0.9949,0.9424,0.9846,0.9298,0.8921,0.9987,0.9987,0.9762,6052.0,732.0,24311.0,8.0
1,Adjusted Vanilla XGB,0.9945,0.9394,0.9840,0.9263,0.8864,0.9992,0.9985,0.9749,6055.0,776.0,24267.0,5.0


# fold 6 

In [30]:
evaluate_model(
    [
        vanilla_xgb,
		
        adjusted_vanilla_xgb,
		
        random_search_tuned_1,
		
		random_search_tuned_2,
		
        grid_tuned_1,
		
        grid_tuned_2,
    ],
    X_test,
    y_test,
    names=[
        "Vanilla XGB",
		
        "Adjusted Vanilla XGB",
		
        "Random Searched Tuned 1",
		
		"Random Searched Tuned 2",
		
        "Grid Tuned Model 1",
		
        "Grid Tuned Model 2",
    ],
	as_table=True
)

,Model Name,AUCPRC,F1,G-Mean,MCC,Precision,Recall,ROCAUC,ACCURACY,TP,FP,TN,FN
0,Vanilla XGB,0.9942,0.9418,0.9838,0.9290,0.8925,0.9969,0.9985,0.9760,6041.0,728.0,24315.0,19.0
2,Random Searched Tuned 1,0.9941,0.9419,0.9840,0.9292,0.8924,0.9974,0.9985,0.9760,6044.0,729.0,24314.0,16.0
4,Grid Tuned Model 1,0.9941,0.9417,0.9839,0.9289,0.8921,0.9972,0.9985,0.9760,6043.0,731.0,24312.0,17.0
5,Grid Tuned Model 2,0.9941,0.9417,0.9840,0.9289,0.8920,0.9974,0.9985,0.9760,6044.0,732.0,24311.0,16.0
3,Random Searched Tuned 2,0.9940,0.9396,0.9839,0.9265,0.8870,0.9988,0.9985,0.9750,6053.0,771.0,24272.0,7.0
1,Adjusted Vanilla XGB,0.5989,0.3306,0.1425,0.0623,0.1980,0.9997,0.5100,0.2111,6058.0,24534.0,509.0,2.0


# fold 7 

In [33]:
evaluate_model(
    [
        vanilla_xgb,
		
        adjusted_vanilla_xgb,
		
        random_search_tuned_1,
		
		random_search_tuned_2,
		
        grid_tuned_1,
		
        grid_tuned_2,
    ],
    X_test,
    y_test,
    names=[
        "Vanilla XGB",
		
        "Adjusted Vanilla XGB",
		
        "Random Searched Tuned 1",
		
		"Random Searched Tuned 2",
		
        "Grid Tuned Model 1",
		
        "Grid Tuned Model 2",
    ],
	as_table=True
)

,Model Name,AUCPRC,F1,G-Mean,MCC,Precision,Recall,ROCAUC,ACCURACY,TP,FP,TN,FN
0,Vanilla XGB,0.9952,0.9455,0.9851,0.9335,0.8985,0.9977,0.9988,0.9776,6045.0,683.0,24361.0,14.0
2,Random Searched Tuned 1,0.9951,0.9435,0.9848,0.9311,0.8943,0.9983,0.9988,0.9767,6049.0,715.0,24329.0,10.0
3,Random Searched Tuned 2,0.9951,0.9428,0.9849,0.9303,0.8925,0.9990,0.9988,0.9764,6053.0,729.0,24315.0,6.0
4,Grid Tuned Model 1,0.9951,0.9436,0.9850,0.9313,0.8942,0.9988,0.9988,0.9768,6052.0,716.0,24328.0,7.0
5,Grid Tuned Model 2,0.9951,0.9440,0.9849,0.9317,0.8952,0.9983,0.9988,0.9769,6049.0,708.0,24336.0,10.0
1,Adjusted Vanilla XGB,0.9947,0.9415,0.9846,0.9288,0.8902,0.9992,0.9983,0.9758,6054.0,747.0,24297.0,5.0


# fold 8 

In [36]:
evaluate_model(
    [
        vanilla_xgb,
		
        adjusted_vanilla_xgb,
		
        random_search_tuned_1,
		
		random_search_tuned_2,
		
        grid_tuned_1,
		
        grid_tuned_2,
    ],
    X_test,
    y_test,
    names=[
        "Vanilla XGB",
		
        "Adjusted Vanilla XGB",
		
        "Random Searched Tuned 1",
		
		"Random Searched Tuned 2",
		
        "Grid Tuned Model 1",
		
        "Grid Tuned Model 2",
    ],
	as_table=True
)

,Model Name,AUCPRC,F1,G-Mean,MCC,Precision,Recall,ROCAUC,ACCURACY,TP,FP,TN,FN
0,Vanilla XGB,0.9946,0.9388,0.9833,0.9255,0.8865,0.9977,0.9986,0.9747,6045.0,774.0,24270.0,14.0
4,Grid Tuned Model 1,0.9946,0.9375,0.9829,0.9239,0.8843,0.9975,0.9986,0.9741,6044.0,791.0,24253.0,15.0
5,Grid Tuned Model 2,0.9946,0.9379,0.9830,0.9243,0.8849,0.9975,0.9986,0.9742,6044.0,786.0,24258.0,15.0
2,Random Searched Tuned 1,0.9945,0.9375,0.9829,0.9239,0.8842,0.9977,0.9986,0.9741,6045.0,792.0,24252.0,14.0
3,Random Searched Tuned 2,0.9945,0.9361,0.9827,0.9223,0.8815,0.9980,0.9986,0.9735,6047.0,813.0,24231.0,12.0
1,Adjusted Vanilla XGB,0.9940,0.9346,0.9825,0.9205,0.8782,0.9987,0.9982,0.9728,6051.0,839.0,24205.0,8.0


# fold 9

In [42]:
evaluate_model(
    [
        vanilla_xgb,
		
        adjusted_vanilla_xgb,
		
        random_search_tuned_1,
		
		random_search_tuned_2,
		
        grid_tuned_1,
		
        grid_tuned_2,
    ],
    X_test,
    y_test,
    names=[
        "Vanilla XGB",
		
        "Adjusted Vanilla XGB",
		
        "Random Searched Tuned 1",
		
		"Random Searched Tuned 2",
		
        "Grid Tuned Model 1",
		
        "Grid Tuned Model 2",
    ],
	as_table=True
)

,Model Name,AUCPRC,F1,G-Mean,MCC,Precision,Recall,ROCAUC,ACCURACY,TP,FP,TN,FN
3,Random Searched Tuned 2,0.9951,0.9429,0.9852,0.9305,0.8921,0.9998,0.9988,0.9764,6058.0,733.0,24311.0,1.0
4,Grid Tuned Model 1,0.9951,0.9432,0.9852,0.9309,0.8928,0.9997,0.9988,0.9766,6057.0,727.0,24317.0,2.0
5,Grid Tuned Model 2,0.9951,0.9446,0.9855,0.9325,0.8955,0.9995,0.9988,0.9772,6056.0,707.0,24337.0,3.0
2,Random Searched Tuned 1,0.9950,0.9435,0.9854,0.9313,0.8932,0.9998,0.9988,0.9767,6058.0,724.0,24320.0,1.0
0,Vanilla XGB,0.9949,0.9460,0.9856,0.9341,0.8986,0.9987,0.9987,0.9778,6051.0,683.0,24361.0,8.0
1,Adjusted Vanilla XGB,0.9949,0.9416,0.9847,0.9289,0.8898,0.9997,0.9987,0.9758,6057.0,750.0,24294.0,2.0


# fold 10

In [5]:
evaluate_model(
    [
        vanilla_xgb,
		
        adjusted_vanilla_xgb,
		
        random_search_tuned_1,
		
		random_search_tuned_2,
		
        grid_tuned_1,
		
        grid_tuned_2,
    ],
    X_test,
    y_test,
    names=[
        "Vanilla XGB",
		
        "Adjusted Vanilla XGB",
		
        "Random Searched Tuned 1",
		
		"Random Searched Tuned 2",
		
        "Grid Tuned Model 1",
		
        "Grid Tuned Model 2",
    ],
	as_table=True
)

,Model Name,AUCPRC,F1,G-Mean,MCC,Precision,Recall,ROCAUC,ACCURACY,TP,FP,TN,FN
0,Vanilla XGB,0.9945,0.9397,0.9834,0.9265,0.8883,0.9974,0.9986,0.9751,6044.0,760.0,24283.0,16.0
2,Random Searched Tuned 1,0.9945,0.9380,0.9834,0.9246,0.8843,0.9987,0.9986,0.9743,6052.0,792.0,24251.0,8.0
3,Random Searched Tuned 2,0.9945,0.9379,0.9836,0.9245,0.8836,0.9993,0.9986,0.9742,6056.0,798.0,24245.0,4.0
4,Grid Tuned Model 1,0.9945,0.9386,0.9835,0.9253,0.8857,0.9983,0.9986,0.9746,6050.0,781.0,24262.0,10.0
5,Grid Tuned Model 2,0.9945,0.9388,0.9835,0.9255,0.8860,0.9982,0.9986,0.9746,6049.0,778.0,24265.0,11.0
1,Adjusted Vanilla XGB,0.9937,0.9365,0.9832,0.9228,0.8811,0.9993,0.9982,0.9736,6056.0,817.0,24226.0,4.0


# fold 10

In [10]:
evaluate_model(
    [
        vanilla_xgb,
		
        adjusted_vanilla_xgb,
		
        random_search_tuned_1,
		
		random_search_tuned_2,
		
        grid_tuned_1,
		
        grid_tuned_2,
    ],
    X_test,
    y_test,
    names=[
        "Vanilla XGB",
		
        "Adjusted Vanilla XGB",
		
        "Random Searched Tuned 1",
		
		"Random Searched Tuned 2",
		
        "Grid Tuned Model 1",
		
        "Grid Tuned Model 2",
    ],
	as_table=True
)

,Model Name,AUCPRC,ROCAUC,F1,G-Mean,MCC,ACCURACY,TP,FP,TN,FN,Precision,Recall
2,Random Searched Tuned 1,0.9946,0.9986,0.9386,0.9835,0.9253,0.9746,6051.0,782.0,24261.0,9.0,0.885555,0.998515
3,Random Searched Tuned 2,0.9946,0.9986,0.9377,0.9835,0.9242,0.9741,6055.0,800.0,24243.0,5.0,0.883297,0.999175
5,Grid Tuned Model 2,0.9946,0.9986,0.9388,0.9835,0.9255,0.9747,6049.0,777.0,24266.0,11.0,0.886171,0.998185
0,Vanilla XGB,0.9945,0.9986,0.9401,0.9836,0.9269,0.9752,6045.0,756.0,24287.0,15.0,0.888840,0.997525
4,Grid Tuned Model 1,0.9945,0.9986,0.9387,0.9836,0.9254,0.9746,6051.0,781.0,24262.0,9.0,0.885685,0.998515
1,Adjusted Vanilla XGB,0.8579,0.9636,0.8516,0.9564,0.8238,0.9322,6051.0,2100.0,22943.0,9.0,0.742363,0.998515


In [11]:
# path_to_models = './saved_models/XGB/'

# # save best model
# random_search_tuned_1.save_model(path_to_models + 'XGB_RandOver_KDD_rs_tuned_1.json')

# # save best spe model - save_model() not working for SelfPacedEnsembleClassifier
# path_to_spe_model = path_to_models + 'XGB_SPE_KDD_random_search_1.pkl'

# # save best spe model using pickle
# # with open(path_to_spe_model, 'wb') as file:
# #     pickle.dump(random_search_tuned_1_spe, file)

# # print('Models saved successfully')